In [1]:
from tensorflow import keras
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

train_scaled = X_train / 255.0
test_scaled = X_test / 255.0 # 0- 255 값을 0-1 로 변환

model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(10, activation=tf.nn.softmax)
        ])

In [2]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_scaled, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 488us/step - loss: 0.4954 - accuracy: 0.8260
Epoch 2/5
1875/1875 [==============================] - 1s 480us/step - loss: 0.3748 - accuracy: 0.8647
Epoch 3/5
1875/1875 [==============================] - 1s 485us/step - loss: 0.3367 - accuracy: 0.8779
Epoch 4/5
1875/1875 [==============================] - 1s 485us/step - loss: 0.3129 - accuracy: 0.8861
Epoch 5/5
1875/1875 [==============================] - 1s 498us/step - loss: 0.2945 - accuracy: 0.8904


여기서 사용한 손실함수는 ***희소한 범주형 크로스 엔트로피(sparse_categorical_crossentropy)*** 라고 부른다.
#### 의류 아이템은 10개의 카테고리 중 하나에 속해있기 때문에 범주형 손실함수를 쓰는 것이 좋음

In [5]:
model.evaluate(test_scaled, y_test)

313/313 [==============================] - 0s 336us/step - loss: 0.3453 - accuracy: 0.8756


[0.34525954723358154, 0.8755999803543091]

### 모델 출력 살펴보기

In [6]:
classifications = model.predict(train_scaled)
print(classifications[0])
print(y_test[0])

1875/1875 [==============================] - 1s 290us/step
[2.7823133e-10 1.9404450e-11 1.1548439e-12 5.6054489e-14 4.4181885e-12
 6.6701439e-05 8.3308941e-12 1.5865797e-03 1.3825446e-08 9.9834669e-01]
9


9번째 레이블을 보면 9.98 * 10^-1 으로 99프로 확률이므로 정확히 예측

### 훈련 조기 종료

In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.95):
            print('\n 정확도 95%에 도달하여 훈련을 멈춥니다!')
            self.model.stop_training = True

1. myCallback 클래스를 만든다
2. 이 클래스는 tf.keras.callbacks.Callback 클래스(부모) 로부터 상속받는다
3. myCallback 클래스 안에 on_epoch_end 함수를 정의
4. 이 함수로 에폭에 대한 자세한 로그 제공, 이 로그에는 정확도가 포함되어 있어 이 값이 95보다 큰지를 확인 가능
5. self.model.stop_training = True로 지정해 훈련 중지

In [9]:
callbacks = myCallback()
mnist = keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

myCallback 클래스를 정의하고 이 클래스의 인스턴스 callbacks 객체를 만든다.

In [11]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))          

In [12]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, callbacks=[callbacks])

Epoch 1/50
1875/1875 [==============================] - 1s 586us/step - loss: 0.4990 - accuracy: 0.8234
Epoch 2/50
1875/1875 [==============================] - 1s 500us/step - loss: 0.3741 - accuracy: 0.8661
Epoch 3/50
1875/1875 [==============================] - 1s 505us/step - loss: 0.3373 - accuracy: 0.8776
Epoch 4/50
1875/1875 [==============================] - 1s 488us/step - loss: 0.3134 - accuracy: 0.8848
Epoch 5/50
1875/1875 [==============================] - 1s 492us/step - loss: 0.2973 - accuracy: 0.8902
Epoch 6/50
1875/1875 [==============================] - 1s 490us/step - loss: 0.2831 - accuracy: 0.8957
Epoch 7/50
1875/1875 [==============================] - 1s 489us/step - loss: 0.2699 - accuracy: 0.9002
Epoch 8/50
1875/1875 [==============================] - 1s 493us/step - loss: 0.2577 - accuracy: 0.9038
Epoch 9/50
1875/1875 [==============================] - 1s 491us/step - loss: 0.2476 - accuracy: 0.9079
Epoch 10/50
1875/1875 [==============================] - 1s 496u

fit 할 때 callbacks에 리스트로 전달

이 사진 이미지는 모두 흑백이고 의류 아이템이 사진 중앙에 놓여있다. 시작하기엔 좋지만 매우 통제된 예제이다.

비전 작업을 더 잘 수행하려면 컴퓨터가 원본 픽셀 대신에 이미지의 특징을 학습해야 한다.

이때 ***합성곱*** 을 사용하면 된다. 다음 장에서 배워보겠다.